In [78]:
import pandas as pd
import numpy as np
import spacy
import glob
import os
import re
import csv

In [7]:
!ls ../financial-news-dataset/ReutersNews106521/20061020

businesspro-energy-exxonmobil-alaska-dc-idUSN2021166720061020
businesspro-google-dc-idUSN2036351320061020


In [30]:
reuters_path = '../financial-news-dataset/ReutersNews106521/'

In [10]:
example_path = '../financial-news-dataset/ReutersNews106521/20061020/businesspro-energy-exxonmobil-alaska-dc-idUSN2021166720061020'

In [11]:
!cat {example_path}

-- Exxon Mobil offers plan to end Alaska dispute
-- By  Yereth Rosen
-- Fri Oct 20, 2006 6:15pm EDT
-- http://www.reuters.com/article/2006/10/20/businesspro-energy-exxonmobil-alaska-dc-idUSN2021166720061020

 

 ANCHORAGE, Alaska  (Reuters) - Exxon Mobil ( XOM.N ) offered 
to pay a $20 million fine and relinquish some leases to the 
state of Alaska to resolve a dispute over a North Slope oil and 
gas field that has remained undeveloped for decades, according 
to a letter made public on Friday. 

  In a proposal sent earlier this week to the Alaska 
Department of Natural Resources, Exxon Mobil offered to pay the 
fine and surrender about 20,000 acres of the 116,000-acre Point 
Thomson unit for failure to develop the field as quickly as 
promised.  The company's offer was accompanied by an updated plan that 
links future field development to a yet-to-be-built pipeline 
that would ship North Slope natural gas to the lower 48 U.S. 
states. The natural gas pipeline, a massive project estima

# Concat all files into one file

## Extract title, author, date, url, location, text

In [32]:
for folder in glob.glob(os.path.join(reuters_path,'*')):
    for file in glob.glob(os.path.join(folder,'*')):
        files.append(file)

In [207]:
def parse_text_components(file):
    dashes = re.compile('-+\s+')
    with open(file) as f:
        title = dashes.sub('', f.readline()).replace('\n','').replace('\t','')
        author = dashes.sub('', f.readline()).replace('\n','').replace('\t','')
        author = re.sub('By\s+','',author)
        date = dashes.sub('', f.readline()).replace('\n','').replace('\t','')
        url = dashes.sub('', f.readline()).replace('\n','').replace('\t','')
        content = f.read().replace('\n',' ').replace('\t','').strip()
        first_dash = content.find('-')  # Find first dash
        location = content[:first_dash].strip()  # Location is everything before
        content = content[first_dash+1:].strip()  # Content is everything after
        location = location.replace('(Reuters)','').strip()  # Take out Reuters tag
        securities = ','.join(re.findall('\(\s([A-Z\d]+.[A-Z]+)\s\)',content))  # Extract securities
        content = re.sub('\(\s[A-Z\d]+.[A-Z]+\s\)','',content).replace('  ',' ')  # Remove securities from content
    return [title, author, date, url, location, securities, content]

In [214]:
with open('all_reuters.txt', "w") as csv_file:
    writer = csv.writer(csv_file, delimiter='\t')
    writer.writerow('title, author, date, url, location, securities, content'.split(', '))
    for i, file in enumerate(files):
        if i%1e4 == 0: print(i)
        try:
            row = parse_text_components(file)
        except:
            continue
        writer.writerow(row)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000


In [217]:
!ls -lh all_reuters.txt

-rw-r--r--  1 ray  staff   319M Apr  4 21:37 all_reuters.txt


In [218]:
!wc -l all_reuters.txt

  106520 all_reuters.txt


In [216]:
with open('all_reuters.txt', "r") as csv_file:
    csvReader = csv.reader(csv_file, delimiter='\t')
    for i in range(5):
        print(next(csvReader)[6])

content
Hitachi Ltd. said on Monday it has agreed with General Electric Co. to expand their global alliance in the nuclear power business, aiming to strengthen their position in a growing market.  The move comes a month after France's Areva CEPFi.PA, the world's largest maker of nuclear reactors, and Japan's Mitsubishi Heavy Industries said they would cooperate in this sector, while Toshiba Corp. completed a $4.2 billion deal to take control of Westinghouse, the U.S. power plant unit of British Nuclear Fuels. "We aim to grow in the global nuclear power market, such as in advanced boiling-water reactors, by generating synergies with our expertise and experience," Hitachi said in a statement. Hitachi, which estimates that 100 more nuclear power plants will be built industry-wide in the next 20 years, said the two companies would form joint ventures in Japan and the United States to construct and maintain nuclear power plants. Hitachi said it would hold a news conference at 5 p.m. (3:00 a